In [3]:
# ✅ 0. Kaggle 데이터 다운로드 설정
!pip install --upgrade kaggle --force-reinstall -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.33.0 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.0 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.33.0 which is incompatible.


In [4]:
from google.colab import files
files.upload()  # kaggle.json 파일 직접 업로드

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"icdi050227","key":"6738515dc0795c218ea602dfa0ddc433"}'}

In [5]:
!mkdir -p ~/.kaggle
!mkdir -p /root/.config/kaggle
!cp kaggle.json ~/.kaggle/
!cp kaggle.json /root/.config/kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!chmod 600 /root/.config/kaggle/kaggle.json

In [7]:
# ✅ Kaggle 데이터 다운로드
!kaggle competitions download -c state-farm-distracted-driver-detection -q


In [8]:
!unzip -q /content/state-farm-distracted-driver-detection.zip -d /content/statefarm_data

replace /content/statefarm_data/driver_imgs_list.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: -o
error:  invalid response [-o]
replace /content/statefarm_data/driver_imgs_list.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [9]:
!unzip -oq /content/state-farm-distracted-driver-detection.zip -d /content/statefarm_data


In [10]:
# ============================================================
# ✅ 1. 라이브러리 설치 및 GPU 설정
# ============================================================
!pip install timm -q

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
from sklearn.model_selection import KFold

In [11]:
# ============================================================
# ✅ 2. GPU 확인
# ============================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')
if torch.cuda.is_available():
    print(f'GPU 이름: {torch.cuda.get_device_name(0)}')

사용 디바이스: cuda
GPU 이름: Tesla T4


In [12]:
# ============================================================
# ✅ 3. 데이터 경로 설정 (Kaggle 다운로드 기준)
# ============================================================
base_dir = "/content/statefarm_data"
driver_csv_path = os.path.join(base_dir, "driver_imgs_list.csv")
train_dir = os.path.join(base_dir, "imgs", "train")
test_dir = os.path.join(base_dir, "imgs", "test")

print("CSV 존재?", os.path.exists(driver_csv_path))
print("Train 폴더 존재?", os.path.exists(train_dir))
print("Test 폴더 존재?", os.path.exists(test_dir))

CSV 존재? True
Train 폴더 존재? True
Test 폴더 존재? True


In [24]:
# ============================================================
# ✅ 4. 하이퍼파라미터
# ============================================================
img_size = 224
batch_size = 64
num_classes = 10
num_epochs = 40
learning_rate = 0.001
num_workers = 2

In [14]:

# ============================================================
# ✅ 5. 데이터 로드
# ============================================================
driver_df = pd.read_csv(driver_csv_path)
all_drivers = sorted(driver_df['subject'].unique())
n_folds = 5
kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)

In [15]:
# ============================================================
# ✅ 6. Dataset 정의
# ============================================================
class DriverDataset(Dataset):
    def __init__(self, data_dir, driver_df, driver_list, transform=None, is_test=False):
        self.data_dir = data_dir
        self.transform = transform
        self.is_test = is_test
        self.images, self.labels = [], []

        if is_test:
            for img_name in os.listdir(data_dir):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self.images.append(os.path.join(data_dir, img_name))
        else:
            driver_subset = driver_df[driver_df['subject'].isin(driver_list)]
            for _, row in driver_subset.iterrows():
                class_name = row['classname']
                img_name = row['img']
                img_path = os.path.join(data_dir, class_name, img_name)
                self.images.append(img_path)
                self.labels.append(int(class_name[1:]))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.is_test:
            return image, os.path.basename(self.images[idx])
        else:
            return image, self.labels[idx]


In [16]:
# ============================================================
# ✅ 7. Transform 설정
# ============================================================
transform_train = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_eval = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [17]:
# ============================================================
# ✅ 8. 학습 / 검증 함수
# ============================================================
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for inputs, labels in tqdm(loader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
    return total_loss/total, 100*correct/total

def validate(model, loader, criterion):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Validating", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)
    return total_loss/total, 100*correct/total

In [27]:
# --------------------------------------------------
# Fold 학습 함수 (✅ 자동 저장 기능 포함)
# --------------------------------------------------
def train_fold(fold_idx, train_drivers, val_drivers):
    print(f"\n========== Fold {fold_idx}/{n_folds} ==========")
    train_dataset = DriverDataset(train_dir, driver_df, train_drivers, transform_train)
    val_dataset = DriverDataset(train_dir, driver_df, val_drivers, transform_eval)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # ✅ timm → torchvision (AlexNet)
    from torchvision import models
    model = models.alexnet(pretrained=True)
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_val_loss = float('inf')
    model_path = f'alexnet_fold{fold_idx}_best.pth'

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_acc = validate(model, val_loader, criterion)
        print(f'Train Loss {train_loss:.4f}, Acc {train_acc:.2f}% | Val Loss {val_loss:.4f}, Acc {val_acc:.2f}%')

        # 🔹 학습 이력 기록
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        # 🔹 5 epoch마다 임시 저장
        if (epoch + 1) % 5 == 0:
            temp_path = f'alexnet_fold{fold_idx}_epoch{epoch+1}.pth'
            torch.save(model.state_dict(), temp_path)
            print(f"🟢 {epoch+1} epoch까지 임시 저장 완료 → {temp_path}")

        # 🔹 최적 모델 저장 (Val Loss 기준)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_path)
            print(f"✅ 새로운 최고 모델 갱신! {model_path} (Val Loss={val_loss:.4f})")

    # 🔹 최종 모델 저장
    final_model_path = f'alexnet_fold{fold_idx}_final.pth'
    torch.save(model.state_dict(), final_model_path)
    print(f"\n💾 학습 완료! 최종 모델 저장: {final_model_path}")

    # 🔹 학습 이력 CSV 저장
    import pandas as pd
    history_df = pd.DataFrame(history)
    history_csv = f'alexnet_fold{fold_idx}_history.csv'
    history_df.to_csv(history_csv, index=False)
    print(f"📈 학습 이력 저장 완료: {history_csv}")

    return model_path, history


In [29]:
import os

print("Train dir:", train_dir)
print("샘플 이미지 경로 5개 확인:")
for i in range(5):
    path = os.path.join(train_dir, "c0", os.listdir(os.path.join(train_dir, "c0"))[i])
    print(path, "존재?", os.path.exists(path))


Train dir: /content/statefarm_data/imgs/train
샘플 이미지 경로 5개 확인:
/content/statefarm_data/imgs/train/c0/img_32280.jpg 존재? True
/content/statefarm_data/imgs/train/c0/img_95596.jpg 존재? True
/content/statefarm_data/imgs/train/c0/img_99342.jpg 존재? True
/content/statefarm_data/imgs/train/c0/img_20727.jpg 존재? True
/content/statefarm_data/imgs/train/c0/img_498.jpg 존재? True


In [31]:
sample_dataset = DriverDataset(train_dir, driver_df, [all_drivers[0]], transform_eval)
sample_loader = DataLoader(sample_dataset, batch_size=8, shuffle=True)

for images, labels in sample_loader:
    print("라벨 dtype:", labels.dtype)
    print("라벨 샘플:", labels)
    break


라벨 dtype: torch.int64
라벨 샘플: tensor([3, 5, 7, 4, 5, 9, 1, 2])


In [28]:
# ============================================================
# ✅ 10. Fold 2 학습 예시 (자동 저장 기능 포함)
# ============================================================

# Fold 정보 설정
fold_info = list(kfold.split(all_drivers))[1]
train_drivers = [all_drivers[i] for i in fold_info[0]]
val_drivers = [all_drivers[i] for i in fold_info[1]]

# 모델 학습 및 저장 함수 실행
model_path, history = train_fold(2, train_drivers, val_drivers)








========== Fold 2/5 ==========

Epoch 1/40


Train Loss 2.3103, Acc 10.45% | Val Loss 2.2984, Acc 10.18%
✅ 새로운 최고 모델 갱신! alexnet_fold2_best.pth (Val Loss=2.2984)

Epoch 2/40


Train Loss 2.3021, Acc 10.68% | Val Loss 2.2957, Acc 11.41%
✅ 새로운 최고 모델 갱신! alexnet_fold2_best.pth (Val Loss=2.2957)

Epoch 3/40


Train Loss 2.3021, Acc 10.29% | Val Loss 2.2994, Acc 11.41%

Epoch 4/40


Train Loss 2.3017, Acc 11.08% | Val Loss 2.2985, Acc 11.41%

Epoch 5/40


Train Loss 2.3015, Acc 10.57% | Val Loss 2.2955, Acc 11.41%
🟢 5 epoch까지 임시 저장 완료 → alexnet_fold2_epoch5.pth
✅ 새로운 최고 모델 갱신! alexnet_fold2_best.pth (Val Loss=2.2955)

Epoch 6/40


Train Loss 2.3016, Acc 10.81% | Val Loss 2.2966, Acc 11.41%

Epoch 7/40


Train Loss 2.3013, Acc 10.91% | Val Loss 2.2979, Acc 11.41%

Epoch 8/40


Train Loss 2.3013, Acc 10.86% | Val Loss 2.2974, Acc 11.41%

Epoch 9/40


KeyboardInterrupt: 

In [25]:
# ============================================================
# ✅ 11. 테스트 예측 + 제출 CSV 생성
# ============================================================
print("\n📢 테스트 데이터 예측 시작...")

test_dataset = DriverDataset(test_dir, driver_df, [], transform_eval, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

model = timm.create_model('alexnet', pretrained=False, num_classes=num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()

predictions, filenames = [], []
with torch.no_grad():
    for images, names in tqdm(test_loader, desc="Predicting"):
        images = images.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        predictions.append(probs.cpu().numpy())
        filenames.extend(names)

preds = np.vstack(predictions)
class_cols = [f'c{i}' for i in range(num_classes)]
submission = pd.DataFrame(preds, columns=class_cols)
submission.insert(0, 'img', filenames)

submission_file = '/content/alexnet_fold2_submission.csv'
submission.to_csv(submission_file, index=False)
print(f"\n✅ 제출 파일 생성 완료: {submission_file}")
print(submission.head())



📢 테스트 데이터 예측 시작...


RuntimeError: Unknown model (alexnet)